In [ ]:
!pip install -U wandb
!pip install tensorflow
!pip install keras

In [ ]:
import requests
url = 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
r = requests.get(url, allow_redirects=True)
open('nature_12K.zip', 'wb').write(r.content)

In [ ]:
!wget 'https://storage.googleapis.com/wandb_datasets/nature_12K.zip'

In [ ]:
!unzip nature_12K.zip

In [5]:
# libraries
import os
import glob
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf 
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import numpy as np 

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
def load_data(input_shape=(256,256)):


    DATA_DIR = r"/content/inaturalist_12K"

    train_dir = os.path.join(DATA_DIR, "train")
    val_dir = os.path.join(DATA_DIR, "val")

    train_generator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=50,
                                    zoom_range=0.2,
                                    shear_range=0.2,
                                    horizontal_flip=True)
    val_generator =ImageDataGenerator(rescale=1./255)

    train_ds = train_generator.flow_from_directory(train_dir, target_size=input_shape, batch_size=128, shuffle=True)
    val_ds = val_generator.flow_from_directory(val_dir, target_size=input_shape, batch_size=128)

    return train_ds, val_ds

# testing out the function
train_ds, val_ds = load_data()

In [7]:
## Utility functions for plotting ##
def plot_sample_images(dir_path):

    subdirs = glob.glob(os.path.join(dir_path,r"*"))

    fig_height = len(subdirs)//5 

    if len(subdirs)%5 != 0:
        fig_height+=1

    fig, axs = plt.subplots(fig_height, 5, figsize=(10, fig_height*2))
    fig.suptitle("Sample images from each class")
    axs = axs.reshape(-1)

    for i, subdir in enumerate(subdirs):

        class_name = os.path.basename(subdir)
        axs[i].set_title(class_name)
        
        img_path = glob.glob(os.path.join(subdir, r"*"))[0]
        img = mpimg.imread(img_path)    
        axs[i].imshow(img)

    plt.show()

In [ ]:
# Plotting samples using the above function
DATA_DIR = r"/content/inaturalist_12K"
plot_sample_images(os.path.join(DATA_DIR,"train"))

In [8]:
class NeuralNet(tf.keras.Model):    

    def __init__(self, base_model, image_shape=(256, 256)):

        super(NeuralNet, self).__init__()

        # instantiating the base model and freezing it's weights
        self.base_model = self.select_model(base_model, image_shape)
        self.base_model.trainable=False

        # The layers below form the classification head 
        self.conv1 = layers.Conv1D(3, 12, 6, activation="relu")
        self.pool1 = layers.MaxPool1D(3,3)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1D(3, 6, 3, activation="relu")
        self.pool2 = layers.MaxPool1D(3,3)
        self.bn2 = layers.BatchNormalization()
        self.output_layer = layers.Dense(10, activation=None)

    @staticmethod
    def select_model(name, image_shape):
        """Selects the pretrained model to be used
        """
        image_shape = list(image_shape)
        image_shape.append(3)
        
        INPUT_SHAPE = tuple(image_shape) 

        if name=="InceptionV3":
            return tf.keras.applications.InceptionV3(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
        elif name=="InceptionResNetV2":
            return tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
        elif name=="ResNet50":
            return tf.keras.applications.ResNet50(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')
        elif name=="Xception":
            return tf.keras.applications.Xception(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet')

    def call(self, x):
        """Performs forward pass for the model"""

        x = layers.Flatten()(self.base_model(x))
        
        x = tf.expand_dims(x, -1)
        x = self.bn1(self.pool1(self.conv1(x)))
        x = self.bn2(self.pool2(self.conv2(x)))
        x = layers.Flatten()(x)

        return self.output_layer(x)

In [9]:
# This is the main function to use to train/fine-tune the model using wandb runs
def train_with_wandb(image_shape=(256, 256), epochs=30, fine_tune_epochs=10):

    config_defaults = {"base_model": "InceptionV3"}

    wandb.init(config=config_defaults, project="Assignment2_PartB", magic=True)

    # 1. Loading the dataset
    train_ds, val_ds = load_data(image_shape)

    # 2. Initializing the model
    model = NeuralNet(wandb.config.base_model, image_shape=image_shape)

    # 3. Compiling the model
    base_learning_rate = 0.0002

    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # 4. Fitting the model
    model.fit(train_ds, 
              validation_data=val_ds,
              epochs=epochs,
              callbacks=[WandbCallback()])
    print("Model trained successfully!!\n")

    # 5. Fine tuning the model
    to_tune_defaults = {
        "InceptionV3": 55,
        "InceptionResNetV2": 55,
        "ResNet50": 50,
        "Xception": 50
    }
    
    model.base_model.trainable = True
    print(f"Total layers in base model is {len(model.base_model.layers)}\n")

    fine_tune_at = len(model.base_model.layers) - to_tune_defaults[wandb.config.base_model]

    for layer in model.base_model.layers[:fine_tune_at]:
        layer.trainable =  False

    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    print("Fine tuning the model ...\n")
    model.fit(train_ds, 
              validation_data=val_ds,
              epochs=fine_tune_epochs,
              callbacks=[WandbCallback()])
    print("Model tuned successfully!!\n")

In [ ]:
wandb.login()
wandb.init(project="Assignment2_PartB", entity="sougatsarangi")

In [11]:
## Setting up the sweep ##
sweep_config = {
  "name": "My Sweep",
  "method": "grid",
  "project": "Assignment2",
  "entity": "sougatsarangi",
  "parameters": {
        "base_model": {
            "values": ["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception"]
        }
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: uvi7chpt
Sweep URL: https://wandb.ai/sougatsarangi/uncategorized/sweeps/uvi7chpt


In [ ]:
# Running the sweep
wandb.agent(sweep_id, function=train_with_wandb)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: f7vxuynb with config:
wandb: 	base_model: InceptionV3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
2. Initializing the model ...

87924736/87910968 [==============================] - 1s 0us/step
3. Compiling the model ...

4. Fitting the model ...

Epoch 1/2
79/79 [==============================] - ETA: 0s - loss: 2.5307 - accuracy: 0.1890 

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


79/79 [==============================] - 1936s 24s/step - loss: 2.5307 - accuracy: 0.1890 - val_loss: 2.1786 - val_accuracy: 0.2025 - _timestamp: 1648899931.0000 - _runtime: 1973.0000
Epoch 2/2
79/79 [==============================] - 1929s 24s/step - loss: 2.0840 - accuracy: 0.3091 - val_loss: 1.9712 - val_accuracy: 0.3140 - _timestamp: 1648901872.0000 - _runtime: 3914.0000
Model trained successfully!!

Total layers in base model is 311

Fine tuning the model ...

Epoch 1/2
79/79 [==============================] - ETA: 0s - loss: 1.7554 - accuracy: 0.4282 

...

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


79/79 [==============================] - 2722s 34s/step - loss: 2.3277 - accuracy: 0.1181 - val_loss: 2.4233 - val_accuracy: 0.1085 - _timestamp: 1648926695.0000 - _runtime: 2768.0000
Epoch 2/2
79/79 [==============================] - 2703s 34s/step - loss: 2.2838 - accuracy: 0.1339 - val_loss: 2.2964 - val_accuracy: 0.1105 - _timestamp: 1648929398.0000 - _runtime: 5471.0000
Model trained successfully!!

Total layers in base model is 175

Fine tuning the model ...

Epoch 1/2
79/79 [==============================] - 4620s 58s/step - loss: 2.5256 - accuracy: 0.1510 - val_loss: 2.4658 - val_accuracy: 0.1030 - _timestamp: 1648934040.0000 - _runtime: 10113.0000
Epoch 2/2
79/79 [==============================] - 4674s 59s/step - loss: 2.2482 - accuracy: 0.1873 - val_loss: 2.3924 - val_accuracy: 0.1350 - _timestamp: 1648938714.0000 - _runtime: 14787.0000
Model tuned successfully!!




accuracy,▁▃▄█
epoch,▁█▁█
loss,▃▂█▁
val_accuracy,▂▃▁█
val_loss,▆▁█▅
accuracy,0.18732
best_epoch,1
best_val_loss,2.29644
epoch,1
loss,2.24824
val_accuracy,0.135


wandb: Agent Starting Run: srlvb143 with config:
wandb: 	base_model: Xception
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


1. Loading the dataset ...

Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
2. Initializing the model ...

83697664/83683744 [==============================] - 1s 0us/step
3. Compiling the model ...

4. Fitting the model ...

Epoch 1/2
61/79 [======================>.......] - ETA: 9:53 - loss: 2.4543 - accuracy: 0.1796 